In [131]:
## http://example.webscraping.com/places/default/search
## 대한민국 찾아서 스크래핑
## results 부분이 비어있어서 못가져온다
## form부분에 아무런 메소드 액션이 없어서 못가져온다.
# http://example.webscraping.com/places/ajax/search.json?&search_term=korea&page_size=10&page=0
# 이런식으로 데이터만 왓다갓다 결과를 못가져옴

## 따라서 우리는 get 뒤에있는 url로 보내면 된다.


## 셀레니옴으로 가져올수있다.
## 돔은 안바뀌고 ajax방식으로 데이터만 왓다갓다
#json 가져오면 파이썬 오브젝트로 가져온다
################ html.json 하면 파이썬 객체 받아올수 있따.

In [4]:
from urllib import *
from html import unescape
import requests
import json 
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
import requests
import time
import bs4
from bs4 import BeautifulSoup
import re
import time


def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url, params= param, data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")
    ## 정규식 re.search(".+?\.tistory.com", "http:\\adam24eve.tistory.com/") #짧게매칭되는거 ? 레이지 

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
   # if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
   #     return []
    
   
   # seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
  #  print("-=================================================")
   # print(seed)
  #  print("-=================================================")
   # html = download("get",seed)
   # dom = BeautifulSoup(html.text, "lxml")
    

    return  [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if  _.has_attr("href") and len(_["href"]) >3 ]

##and checkBlog(requests.compat.urljoin(seed, _["href"]))






In [133]:
url = "http://example.webscraping.com/places/default/search"
data = {
    "search_term" : "korea",
    "page_size" : "10",
    "page" : "0"
}
html = download("post", url, data)
dom = BeautifulSoup(html.text, "lxml")

In [134]:
dom.text

' \n\nExample web scraping website\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!--\n    // These variables are used by the web2py_ajax_init function in web2py_ajax.js (which is loaded below).\n    var w2p_ajax_confirm_message = "Are you sure you want to delete this object?";\n    var w2p_ajax_disable_with_message = "Working...";\n    var w2p_ajax_date_format = "%Y-%m-%d";\n    var w2p_ajax_datetime_format = "%Y-%m-%d %H:%M:%S";\n    var ajax_error_500 = \'An error occured, please <a href="/places/default/search?page=0&amp;page_size=10&amp;search_term=korea">reload</a> the page\'\n    //-->\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLog In Sign Up Log In\n\nHomeSearch\n\n\n\n\n\n\n\n\n\n\n                    Example web scraping website\n                    \n\n\n\n\n\n\n\n\n\nName:\n\n\n\nPage size:\n\n\n4\n10\n20\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nvar NUM_COLUMNS = 2;\nvar page = 0;\n\n\n$(function() {\n    $("form").submit(function() {\n        page = 0;\n        search();\n        return false;\n    });\

In [135]:
######### 정답
## ajax방식으로 동작하기 때문에 get이나 post방식으로 동적을 바뀌는 데이터를 가져올수 없다. 
## 결과값이 나오는 엘리먼트가 처음에 돔을 받아올때 비어있기 떄문이다. 

In [24]:
url = "http://example.webscraping.com/places/ajax/search.json"

param = {
    "search_term":"korea",
    "page_size" : "10",
    "page" : "0"
}
    
html = download("get", url, param) ## post 왜 되냐?? re
dom = BeautifulSoup(html.text, "lxml")

#html = download("post", "http://example.webscraping.com/places/default/search",data=data)

In [25]:
html.text

'{"records": [{"pretty_link": "<div><a href=\\"/places/default/view/North-Korea-165\\"><img src=\\"/places/static/images/flags/kp.png\\" /> North Korea</a></div>", "country": "North Korea", "id": 4559097}, {"pretty_link": "<div><a href=\\"/places/default/view/South-Korea-211\\"><img src=\\"/places/static/images/flags/kr.png\\" /> South Korea</a></div>", "country": "South Korea", "id": 4559143}], "num_pages": 1, "error": ""}\n'

In [26]:
url

'http://example.webscraping.com/places/ajax/search.json'

In [16]:
dom.text

'{"records": [{"pretty_link": " North Korea", "country": "North Korea", "id": 4559097}, {"pretty_link": " South Korea", "country": "South Korea", "id": 4559143}], "num_pages": 1, "error": ""}\n'

In [138]:
for _ in html.json()["records"]:
    print(_["country"])

North Korea
South Korea


In [139]:
## 비동기식으로 데이터 가져올려며 이런식으로 godigksek.
## pase re

## 중간에 변경된 돔을 캐치할수 있게 해줌 selenium
## 원래는 브라우져를 자동화 해주는 애 , 웹 서비스 웹시스템을 단위테스트, 특정시나리오 테스트 -> 테스팅을 위해 만들어진애 =
## 사람이 어디 마우스 클릭 했을때 롤업, 드롭다운 이런 액션들 자바스크립트가 하는데 자바스크립트 해석할수있게 비동기식 ajax 해석할수 있게 해줌
## 브라우저를 띄워서 동적으로 변하는 돔을 가져올수 있게 해줌
## cf 애자일방법론이랑 잘맞다



In [140]:
!pip install selenium

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [141]:
## json '' 하면 에러난다 "" 해줘야함

In [142]:
%%writefile account.json
{
    "id" : "legendazi",
    "pw" : "dud0703!"
}

Overwriting account.json


In [143]:
import json
with open("account.json") as fp:
    account = json.load(fp)

In [144]:
account["pw"]

'dud0703!'

In [185]:
from selenium import webdriver

driver = webdriver.Chrome()
# 창끄면 핸들러 잃어버림
# driver.close()



In [168]:
driver.close()

In [186]:
driver.get("http://example.webscraping.com/places/default/search")

In [147]:
## html 받아옴
driver.page_source

'<html class=" js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" lang="en-us" style=""><!--<![endif]--><head>\n<title>Example web scraping website</title>\n  <!--[if !HTML5]>\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n  <![endif]-->\n  <!-- www.phpied.com/conditional-comments-block-downloads/ -->\n  <!-- Always force latest IE rendering engine\n       (even in intranet) & Chrome Frame\n       Remove this if you use the .htaccess -->\n\t   \n  <meta charset="utf-8">\n\n  <!-- http://dev.w3.org/html5/markup/meta.name.html -->\n  <meta name="applicati

In [189]:
inputTag = driver.find_element_by_id("search_term")
inputTag
# driver.find_element_by_css_selector("#search_term")

<selenium.webdriver.remote.webelement.WebElement (session="5b6af686ee7eff3c6e7515faa35c9ba5", element="052d562c-2bd8-41dc-a79d-b0a96ad79ce8")>

In [190]:
inputTag.send_keys("korea")

In [191]:
inputTag2 = driver.find_element_by_xpath(".//input[@type='submit']").click()

In [192]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#results")

# 돔요청안하고 바뀐값 반영된 돔만 그대로 가져올수 있다는게 중요. request안했다.

##어제 브라우져에는 있는데 코드에는없어서 소스코드 찾아봐야됬는데 이거쓰면 됨 re!!
## 렌더링 되기전하면 에러남 기다려주는 시간 필요

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

In [193]:
driver.find_element_by_css_selector("#results").text ##selnium 단에서 끝낼수 있는 일인지
                                                    ## 아니면 뷰티플 소프까지 써서 해야할일인지

'North Korea\nSouth Korea'

In [164]:
#############네이버
driver.get("https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com")


InvalidSessionIdException: Message: invalid session id


In [159]:
inputTag = driver.find_element_by_id("id")
inputTag.send_keys(account["id"])
inputTag2 = driver.find_element_by_id("pw")
inputTag2.send_keys(account["pw"])
inputTag3 = driver.find_element_by_css_selector(".btn_global").click()
## 이런거 캡챠들 자동입력 방지문자 뜬다 로그인 안된다 자동로그인 네이버에서 막아놈 
## 자동입력해주는거 브라우저 계속 새로운 id 네이버에서 id 계속바뀌면 막는다.


In [163]:
driver.get("https://mail.naver.com/read/8162#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%220%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D")

InvalidSessionIdException: Message: invalid session id


In [161]:
print([driver.find_elements_by_css_selector("strong.mail_title")])

[[]]


In [157]:
#inputTag3 = driver.find_element_by_css_selector(".mail_title _ccr(lst.title)._c1(mrCore|clickTitle|8160)._m2(mrCore|middleClickTitle|8160) > span")
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print("[CSS]", _.text)

for _ in  driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print("[XPATH]", _.text)

## 리액트나 앵귤러 js 떄문에 @ 좆같은형식 생겼다.
## xpath 구조로 다가져올수 있다. css 원하느 

In [ ]:
inputTag3.text

In [ ]:
## 스팸메일함 하느거  

## ._d6(mcDragndrop|html5DragOver)._d7(mcDragndrop|html5Drop) 는 왜안되는지 다시 해보기 re!!

In [45]:
inputTag3 = driver.find_element_by_css_selector(".item_wrap.bu6").click()

In [ ]:
## 쿠키값을 세션에 전달해주면 로그인된채로 돌아다닐수 있다.

In [52]:
for cookie in driver.get_cookies():
   # print(cookie["name"],":", cookie["value"])
    session.cookies.set(cookie["name"], cookie["value"])  ## 드라이버에서 받은 쿠키를 세션에 넣음

In [49]:
import requests 
session = requests.Session()   ### 원랴 리퀘스트 객체에서 받은 쿠키

In [53]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='NID_AUT', value='d23UK/NHzAkFxx/dB3ZNdE+DLOoAL6wq0x5RaykVmTL9WHutt0Ice69iWd58v6xs', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='pVhgCXDwRD3DhQ3SYfu4bVpllCoucB0KzyJixmEfa0g=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_SES', value='AAABiGanI/KxvT22nz5DPzNOKDdSUtMrgJHyUd+eM6542lSLVssnjph55lri4T7x8HmRWcWCXwBJ+wfiHvcS+R1+FU5S/UQ3+KOluBo++eF4r33nCRLjD9mDts2ApFotcPI8YgU7v7vVNNb5LSiVVySjTPfgOj90OPXXJpxF42hdABDRhnF1RBRp0KApiKVhRF+J78RSjXjjIkQ8bX6hSNL0ZeZX3Qaa2WYFqPqUWx5nljhHS52SLHFiLPrlcLVm3qchetan

In [56]:
html = session.get("https://mail.naver.com/read/8162#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%220%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D")
html.text

'\n\n\n<!DOCTYPE HTML>\n\n\t\n\t\n\t\n\t\n\t\n\t\t<html lang="ko">\n\t\n\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190611.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190611.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net" />\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190611.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\

In [57]:
## 껍데기돔은 받아오고 남고 데이터는 계속 바뀜  -> 이거를 스틸 -> 섹션객체에 집어넣으면 가져온다

SyntaxError: invalid syntax (<ipython-input-58-51766b13eda7>, line 1)

In [ ]:
##받은메일함

In [184]:
from html import unescape
html = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=legendazi")
html.text
for _ in html.json()["mailData"] :
    print(unescape(_["subject"]))
    
    
## unescapte &lt #나오고 이런 글자 꺠지던거 
## 셀리니엄으로 데이터 가져오면 x 렌더링 시간떄문에  쿠키를 들고오는거 할떄만 셀레니움 쓴다.

'{"currentTime":1563340588,"lastPage":213,"Message":"","pageSize":30,"unreadCount":6132,"mailData":[{"mailSN":8162,"folderSN":0,"folderName":null,"iconType":0,"status":2097157,"from":{"name":"오늘의정보","email":"sunshine4@letter79.co.kr"},"subject":"(광고) 7월 17일, &lt;라식 할인&gt; 안내 드립니다","receivedTime":1563321247,"sentTime":1563321247,"size":7016,"attachCount":0,"iDomainEmail":"legendazi@naver.com","priority":3,"securityLevel":null,"reSend":false,"color":"#ffffff","preview":"","mimeSN":"1563321247.603415.45664.1792","toList":[{"name":"송환의님","email":"legendazi@naver.com"}],"spamType":null,"attachInfo":[],"threadId":"","bccList":[],"receiveByBcc":false,"threadCount":0,"unreadThreadCount":0,"firstLocatedTime":1563321247,"remindAt":0,"threadRemindAt":0,"threadRemindFlag":0,"category":213,"lastLocatedTime":0,"toMe":true,"bigFile":false,"isDomainMail":false,"statusHex":"200005"},{"mailSN":8161,"folderSN":0,"folderName":null,"iconType":0,"status":2097157,"from":{"name":"잇다(itdaa)","email":"hello@itd

In [66]:
html = requests.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=legendazi")
html.text

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [ ]:
## 쿠키값이 없기때문에 똑같은 주소로 리퀘스트 보냈는데 로그인이 안되있다고 나온다.

## ajax 데이터 서버쪽으로 보내는거 무조건 url 붙는다 그 url 찾는게 관건

In [182]:
html = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=legendazi")
html.text

'{"currentTime":1563340270,"lastPage":1,"Message":"","pageSize":30,"unreadCount":3,"mailData":[{"mailSN":8137,"folderSN":5,"folderName":null,"iconType":0,"status":2105344,"from":{"name":"모아의","email":"moa-noreply@giftmail.co.kr"},"subject":"(광고) 나에게맞는 가발/모발이식 은 무엇? 신청만해도 300만원지원","receivedTime":1562919690,"sentTime":1562919688,"size":8013,"attachCount":0,"iDomainEmail":"legendazi@naver.com","priority":3,"securityLevel":null,"reSend":false,"color":"#ffffff","preview":"","mimeSN":"1562919690.35387.51253.14080","toList":[{"name":"고객님","email":"legendazi@naver.com"}],"spamType":"SYSTEM_DEFINE","attachInfo":[],"threadId":"","bccList":[],"receiveByBcc":false,"threadCount":0,"unreadThreadCount":0,"firstLocatedTime":1562919690,"remindAt":0,"threadRemindAt":0,"threadRemindFlag":0,"category":-1,"lastLocatedTime":0,"toMe":true,"bigFile":false,"isDomainMail":false,"statusHex":"202000"},{"mailSN":8132,"folderSN":5,"folderName":null,"iconType":0,"status":2105344,"from":{"name":"교정할인정보","email":"sun1

In [ ]:
### 스팸메일함

In [76]:
from html import unescape
html = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=legendazi")
html.text
for _ in html.json()["mailData"] :
    print(unescape(_["subject"]))
    
    
## unescapte &lt #나오고 이런 글자 꺠지던거 
## 셀리니엄으로 데이터 가져오면 x 렌더링 시간떄문에  쿠키를 들고오는거 할떄만 셀레니움 쓴다.
# folderSN만 다르다 이거가지고 겟방식으로 포스트 보냈는데 이거만 다르게해서 데이터 가져오고 이런거 할수 있다.

'{"currentTime":1563338366,"lastPage":1,"Message":"","pageSize":30,"unreadCount":3,"mailData":[{"mailSN":8137,"folderSN":5,"folderName":null,"iconType":0,"status":2105344,"from":{"name":"모아의","email":"moa-noreply@giftmail.co.kr"},"subject":"(광고) 나에게맞는 가발/모발이식 은 무엇? 신청만해도 300만원지원","receivedTime":1562919690,"sentTime":1562919688,"size":8013,"attachCount":0,"iDomainEmail":"legendazi@naver.com","priority":3,"securityLevel":null,"reSend":false,"color":"#ffffff","preview":"","mimeSN":"1562919690.35387.51253.14080","toList":[{"name":"고객님","email":"legendazi@naver.com"}],"spamType":"SYSTEM_DEFINE","attachInfo":[],"threadId":"","bccList":[],"receiveByBcc":false,"threadCount":0,"unreadThreadCount":0,"firstLocatedTime":1562919690,"remindAt":0,"threadRemindAt":0,"threadRemindFlag":0,"category":-1,"lastLocatedTime":0,"bigFile":false,"toMe":true,"statusHex":"202000","isDomainMail":false},{"mailSN":8132,"folderSN":5,"folderName":null,"iconType":0,"status":2105344,"from":{"name":"교정할인정보","email":"sun1

In [178]:
html = session.get("https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=10&articleCount=3&useMyCafeEvent=true")
html.text

'{"message":{"status":"200","error":{"code":"","msg":""},"result":{"myCafe":{"cafes":[{"cafeId":19543191,"cafeName":"리그오브레전드 한국커뮤니티 - LoLKor","cafeUrl":"lolkor","articleNewCounts":13414,"formattedArticleNewCounts":"13,414","cafeThumbnailPcUrl":"https://cafeptthumb-phinf.pstatic.net/MjAxNzA1MjJfOTAg/MDAxNDk1NDQ1MTExNjY1.ZnszyoIeRE4hdE-WQxvxtmaXjVqqHHNGX_24jv0QTPsg.wnsuY9L-DPxeuZxJc0QHU2lLP4U-kSkcitaIjmezO8Qg.PNG.pitro/200x200.png?type=f72_72_mask","cafeThumbnailMobileUrl":"https://mcafethumb-phinf.pstatic.net/MjAxNzA1MjJfOTAg/MDAxNDk1NDQ1MTExNjY1.ZnszyoIeRE4hdE-WQxvxtmaXjVqqHHNGX_24jv0QTPsg.wnsuY9L-DPxeuZxJc0QHU2lLP4U-kSkcitaIjmezO8Qg.PNG.pitro/200x200.png?type=f138_138_mask","manageCafe":false,"dormantCafe":false,"powerCafe":false,"starJoinCafe":false,"gameCafe":false,"educationCafe":false,"mobileCafeName":"LoLKor","aheadOfTime":"6초 전","lastUpdateDate":"2019-07-17 14:10:21","lastVisitDate":"2019-07-10 01:25:42","openType":"O","favoriteCafe":false,"hasNewArticle":true,"articles":[{"cafe

In [ ]:
## 내가 가입되어있는 카페명 들고오기
## html.json(), html.text() 여태까지 했던거 text인데, 갑자기json? 구분 re!!

In [172]:
html = session.get("https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=10&articleCount=3&useMyCafeEvent=true")
for _ in html.json()["message"]["result"]["myCafe"]["cafes"]:
    print(_["cafeName"])
print("ssssssssss")


리그오브레전드 한국커뮤니티 - LoLKor
스펙업 | 대학생,대외활동,공모전,알바,인턴,취업,NCS,공기업
[비트맨] 비트코인 NO.1 커뮤니티 - 가상화폐/블록체인/정보공유
공취사│공기업,공공기관,취업,NCS,자기소개서,면접,필기,채용
공준모★공기업채용,공사공단취업,교직원,NCS필기,한국사,대기업
중고나라
[공기업을 준비하는 사람들의 모임] NCS,취업,채용,스펙,공준모
독취사-취업,대학생,대기업,공기업,NCS,인턴,공모전,승무원,토익
수만휘-수능날만점시험지를휘날리자★수능공부과외편입유학
토익캠프-TOEIC토익스피킹공부/영어회화/정답/오픽OPIc시험/토스
ssssssssss


In [173]:
driver.get("https://section.cafe.naver.com/")

InvalidSessionIdException: Message: invalid session id


In [170]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="2c3d5165ce21de58c819ce8f6a4ac45d")>

In [171]:
len(driver.find_elements_by_xpath("//div[@class='user_mycafe_list']//div[@class='user_mycafe_info']/a/strong[@class='name']"))

InvalidSessionIdException: Message: invalid session id


In [175]:
driver.get("https://www.naver.com/")

InvalidSessionIdException: Message: invalid session id


In [174]:


for _ in driver.find_elements_by_xpath("//div[@class='user_mycafe_list']//div[@class='user_mycafe_info']/a/strong[@class='name']"):
    print(_.text)

InvalidSessionIdException: Message: invalid session id


In [ ]:
f